In [1]:
import pandas as pd

# The Parser module is used to convert the raw data to a standard format
from mobvis.preprocessing.parser import Parser as par

# The Locations module is used to find the Geo-locations of the trace, used by almost all metrics
from mobvis.metrics.utils.Locations import Locations as loc
# The HomeLocations module is used to find the Home-locations of the trace, used by some metrics
from mobvis.metrics.utils.HomeLocations import HomeLocations as hloc
# The Contacts module is used to detect the Contacts between the nodes, used by Social metrics
from mobvis.metrics.utils.Contacts import Contacts as cnt

# The MetricBuilder module can be used to instantiate all the metrics 
from mobvis.metrics.utils.MetricBuilder import MetricBuilder as mb

# The metric and spatial plotters contains all the visualizations of MobVis
from mobvis.plots.metric_plotter import *
from mobvis.plots.spatial_plotter import *

In [ ]:
traceRoot = "./traces/"
traceName = "aSIOTmm_30-07-2024_18.44.34.013/" # EDIT HERE
# Add the path to the unfixed info file
infoUnfixedH = pd.read_csv(traceRoot + traceName + "info.csv", delimiter=",", names=["id", "owner", "type"], skiprows=1)
# Add the path to the unfixed contacts_trace file
siotContactsTraceH = pd.read_csv(traceRoot + traceName + "contacts_trace.csv", delimiter=",", names=["id1", "id2", "tStart", "tEnd", "id1_start_pos_x", "id1_start_pos_y", "id2_start_pos_x", "id2_start_pos_y", "id1_end_pos_x", "id1_end_pos_y", "id2_end_pos_x", "id2_end_pos_y"], skiprows=1)


In [ ]:
def getIDsOfHumanNodes(df):
    df['type'] = df['type'].str.strip().str.lower()
    humanNodes = df[df['type'] == 'humannode']
    return humanNodes["id"].unique()


In [ ]:
infoUnfixedH['type'] = infoUnfixedH['type'].str.strip().str.lower()

humanNodesH = getIDsOfHumanNodes(infoUnfixedH)

copy = siotContactsTraceH
# print(copy)
filtered = copy
filtered = filtered[~filtered['id1'].isin(humanNodesH)]
filtered = filtered[~filtered['id2'].isin(humanNodesH)]

filtered['set'] = filtered.apply(lambda x: frozenset({x['id1'], x['id2']}), axis=1)


# drop duplicates
noDuplicates = filtered.drop_duplicates(subset=['tStart', 'tEnd', 'set'])
# noDuplicates
    # df = df.drop(columns=['set'])
noDuplicates.to_csv(traceRoot + traceName + "contacts_traceNoHumansNoDups.csv", index=False, header=True)

In [ ]:
# Fix Contacts
infoFixedCSVL = pd.read_csv(traceRoot + traceName + "infoFixed.csv", delimiter=",", names=["id", "owner", "type", "old_id"], skiprows=1)
id_mappingL = dict(zip(infoFixedCSVL['old_id'], infoFixedCSVL['id']))


    # Replace old IDs with new IDs using the mapping
noDuplicates['id1'] = noDuplicates['id1'].map(id_mappingL)
noDuplicates['id2'] = noDuplicates['id2'].map(id_mappingL)
    # convert id to int
noDuplicates['id1'] = noDuplicates['id1'].astype(int)
noDuplicates['id2'] = noDuplicates['id2'].astype(int)
noDuplicates = noDuplicates.drop(columns=['set'])
noDuplicates.to_csv(traceRoot + traceName + "contacts_traceNoHumansNoDupsFixed.csv", index=False, header=True)